# Board Game Geek Ratings

Load packages

In [1]:
import pandas as pd

# ignore warnings
import warnings
warnings.filterwarnings('ignore') 

# pandas defaults
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)

Load data (from https://www.kaggle.com/jvanelteren/boardgamegeek-reviews)

In [2]:
bg = pd.read_csv('games_detailed_info.csv')

# keep relevant fields
bg_clean = bg[['id', 'primary', 'description', 'yearpublished', 'minplayers', 'maxplayers',
               'suggested_num_players', 'suggested_playerage', 'suggested_language_dependence', 'playingtime', 
               'boardgamecategory', 'boardgamemechanic', 'boardgamedesigner', 'usersrated', 'average', 'bayesaverage',
               'stddev', 'owned', 'wanting', 'wishing', 'numcomments', 'numweights', 'averageweight', 'Board Game Rank', 
               'Strategy Game Rank', 'Family Game Rank', 'Party Game Rank', 'Abstract Game Rank', 'Thematic Rank', 
               'War Game Rank', 'Customizable Rank']].sort_values('usersrated', ascending=False).reset_index()

# keep only top 1000 most rated games
bg_clean = bg_clean[bg_clean['index']<1000]

Avoid duplicate board game names

In [3]:
# get all games
list_bg = bg_clean[['primary','yearpublished']]

# shows with unique names
only_unique = list_bg.drop_duplicates(['primary'], keep=False)
only_unique['board_game'] = only_unique['primary']

# shows with duplicated names
duplicated = list_bg[list_bg.duplicated(['primary'], keep=False)]
duplicated['board_game'] = duplicated['primary']+' ('+duplicated['yearpublished'].astype(str)+')'

# join all data together
new_names = pd.concat([only_unique, duplicated])
games = pd.merge(bg_clean, new_names, on=['primary', 'yearpublished'])

Parse fields that have multiple values and put it in columns

In [4]:
type = pd.melt(bg_clean,id_vars=['index'], value_vars=['Strategy Game Rank','Family Game Rank',
                                                       'Party Game Rank','Abstract Game Rank','Thematic Rank',
                                                       'War Game Rank','Customizable Rank'],
            var_name='var_new_name',value_name='value_new_name')
type = type[type['value_new_name'].notna()]
type['type'] = type['var_new_name'].str.replace(" Game Rank" ,'').str.replace("Rank" ,'')
type['type'] = type['type'].str.strip()
type = type[['index', 'type']]

In [5]:
designer = bg_clean['boardgamedesigner'].str.split("',", expand=True).reset_index()
designer = pd.melt(designer,id_vars=['index'], value_vars=[0, 1, 2, 3, 4, 5, 6, 7],
            var_name='var_new_name',value_name='value_new_name')
designer = designer[designer['value_new_name'].notna()]
designer['value_new_name'] = designer['value_new_name'].str.replace("'" ,'').str.replace("[" ,'').str.replace("]" ,'').str.replace('"' ,'')
designer['designer'] = designer['value_new_name'].str.strip()
designer = designer[['index', 'designer']]

In [6]:
category = bg_clean['boardgamecategory'].str.split("',", expand=True).reset_index()
category = pd.melt(category,id_vars=['index'], value_vars=[0, 1, 2, 3, 4, 5, 6, 7],
            var_name='var_new_name',value_name='value_new_name')
category = category[category['value_new_name'].notna()]
category['category'] = category['value_new_name'].str.replace("'" ,'').str.replace("[" ,'').str.replace("]" ,'')
category['category'] = category['category'].str.strip()
category = category[['index', 'category']]

# remove uncommon categories
count = category.groupby(['category']).agg(count= ('index', 'count')).reset_index()
category = pd.merge(category, count[count['count']>=15][['category']], on='category')

In [7]:
mechanic = bg_clean['boardgamemechanic'].str.split("',", expand=True).reset_index()
mechanic = pd.melt(mechanic,id_vars=['index'], value_vars=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
            var_name='var_new_name',value_name='value_new_name')
mechanic = mechanic[mechanic['value_new_name'].notna()]
mechanic['value_new_name'] = mechanic['value_new_name'].str.replace("'" ,'').str.replace("[" ,'').str.replace("]" ,'').str.replace('"' ,'')
mechanic['mechanic'] = mechanic['value_new_name'].str.strip() 
mechanic = mechanic[['index', 'mechanic']]

# remove uncommon mechanics
count = mechanic.groupby(['mechanic']).agg(count= ('index', 'count')).reset_index()
mechanic = pd.merge(mechanic, count[count['count']>=15][['mechanic']], on='mechanic')

Join everything (easier to work on Google Data Studio)

In [8]:
games_final = pd.merge(games, type, on='index', how='left')
games_final = pd.merge(games_final, category, on='index', how='left')
games_final = pd.merge(games_final, mechanic, on='index', how='left')
games_final = pd.merge(games_final, designer, on='index', how='left')

# remove unwanted fields
drop = bg[['primary', 'suggested_playerage', 'suggested_language_dependence', 'suggested_num_players',
           'boardgamecategory', 'boardgamemechanic', 'boardgamedesigner', 'description',
           'Strategy Game Rank', 'Family Game Rank', 'Party Game Rank', 
           'Abstract Game Rank', 'Thematic Rank', 'War Game Rank', 'Customizable Rank']]
games_final.drop(drop, axis = 1, inplace=True )

Save data

In [9]:
games_final.to_excel('Board Game Geek.xlsx')
games_final.head()

,index,id,yearpublished,minplayers,maxplayers,playingtime,usersrated,average,bayesaverage,stddev,owned,wanting,wishing,numcomments,numweights,averageweight,Board Game Rank,board_game,type,category,mechanic,designer
0,0,30549,2008,2,4,45,96241,7.61567,7.51795,1.32632,144727,640,8571,15778,5232,2.4148,91,Pandemic,Strategy,NaN,Action Points,Matt Leacock
1,0,30549,2008,2,4,45,96241,7.61567,7.51795,1.32632,144727,640,8571,15778,5232,2.4148,91,Pandemic,Strategy,NaN,Hand Management,Matt Leacock
2,0,30549,2008,2,4,45,96241,7.61567,7.51795,1.32632,144727,640,8571,15778,5232,2.4148,91,Pandemic,Strategy,NaN,Cooperative Game,Matt Leacock
3,0,30549,2008,2,4,45,96241,7.61567,7.51795,1.32632,144727,640,8571,15778,5232,2.4148,91,Pandemic,Strategy,NaN,Set Collection,Matt Leacock
4,0,30549,2008,2,4,45,96241,7.61567,7.51795,1.32632,144727,640,8571,15778,5232,2.4148,91,Pandemic,Strategy,NaN,Point to Point Movement,Matt Leacock
